<a href="https://colab.research.google.com/github/Cellous/ai-bootcamp-portfolio/blob/main/week-01-foundations/notebooks/week01_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("welcome to my AI Agent Lab!")

In [ ]:
!pip install requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install langchain

In [ ]:
pip install openai

In [ ]:
!pip install transfromers

In [ ]:
!pip install transformers

In [ ]:
!pip install numpy pandas

In [ ]:
!pip install matplotlib seaborn

In [ ]:
import openai
import os

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the pre-trained model and tokenizer
model_name = "gpt2"  # Choose a lighter model for resource management
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Make sure the tokenizer has a padding token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))

# Function to generate a response
def generate_response(prompt, max_length=50, temperature=0.7, top_p=0.9):
    try:
        # Encode the prompt and generate response
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=min(len(inputs['input_ids'][0]) + 20, max_length),
            temperature=min(1.0, max(0.2, temperature)),
            top_p=max(0.1, min(0.9, top_p)),
            num_return_sequences=1,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )

        # Decode and format the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.replace(prompt, "").strip()
        # Truncate response at the first sensible stop (e.g., period)
        if '.' in response:
            response = response.split('.')[0] + '.'
        return response
    except RuntimeError as err:
        print(f"Error during generation: {err}")
        return "Sorry, I couldn't process your request due to an internal error."

# Interactive chat session
def chatbot_session():
    print("Bot: Hello! What's your name?")
    user_name = input("You: ")

    while True:
        bot_prompt = f"{user_name}: "
        bot_response = generate_response(bot_prompt, max_length=50)
        print(f"Bot: {bot_response}")

        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Bot: Goodbye!")
            break

        bot_prompt += f"{user_input}\nBot: "

# Start the chatbot session
chatbot_session()